### Extracting suburb using lat long

In [1]:
import pandas as pd
import numpy as np
import ast 
import geopandas as gp
from shapely.geometry import Point, shape

In [6]:
council_file = open("councils.txt", "r")   # Openning the text file in read mode
council = council_file.readlines()         # Converting the content of the text file as a list
council

["BANYULE : ['Abbotsford', 'Airport West', 'Albert Park', 'Alphington']\n",
 "BRIMBANK : ['Altona', 'Altona North', 'Armadale', 'Ascot Vale']\n",
 "DAREBIN : ['Ashburton', 'Ashwood', 'Avondale Heights', 'Balaclava']\n",
 "HOBSONS BAY : ['Balwyn', 'Balwyn North', 'Bentleigh', 'Bentleigh East']\n",
 "HUME : ['Box Hill', 'Braybrook', 'Brighton', 'Brighton East']\n",
 "MARIBYRNONG : ['Brunswick', 'Brunswick West', 'Bulleen', 'Burwood']\n",
 "MELBOURNE : ['Camberwell', 'Canterbury', 'Carlton North', 'Carnegie']\n",
 "MELTON : ['Caulfield', 'Caulfield North', 'Caulfield South', 'Chadstone']\n",
 "MOONEE VALLEY : ['Clifton Hill', 'Coburg', 'Coburg North', 'Collingwood']\n",
 "MORELAND : ['Doncaster', 'Eaglemont', 'Elsternwick', 'Elwood']\n",
 "NILLUMBIK : ['Essendon', 'Essendon North', 'Fairfield', 'Fitzroy']\n",
 "WHITTLESEA : ['Fitzroy North', 'Flemington', 'Footscray', 'Glen Iris']\n",
 "WYNDHAM : ['Glenroy', 'Gowanbrae', 'Hadfield', 'Hampton']\n",
 "YARRA : ['Hampton East', 'Hawthorn', 'H

In [7]:
council_dic = {}     # Creating an empty dictionary
all_suburbs = []     # Creating an empty list

# Iterating over each line of the text file
for each in council:
    k = each[0:each.find(":")].strip()                     # Storing the characters before ':' as 'k' for each line 
    v = ast.literal_eval(each[each.find(":")+1:].strip())  # Storing the characters after ':' as 'v' for each line and converting string representation of list to list
    all_suburbs = all_suburbs+v                            # Storing all suburbs of one council
    council_dic[k] = v                                     # Storing 'k' and 'v' as key and value pairs in dictionary

In [8]:
shape_df = gp.read_file("./VIC_LOCALITY_POLYGON_shp.shp")                           # Reading the .shp file using geopandas
shape_df = shape_df[['VIC_LOCA_2','geometry']]                                      # Fetching the required columns only
shape_df = shape_df[shape_df['VIC_LOCA_2'].isin([x.upper() for x in all_suburbs])]  # Considering only the suburbs present in council file
shape_df.head()

,VIC_LOCA_2,geometry
30,BUNDOORA,"POLYGON ((145.02571603 -37.71385402, 145.02567..."
31,HEIDELBERG WEST,"POLYGON ((145.03327169 -37.74888857, 145.03337..."
43,DEEPDENE,"POLYGON ((145.06135741 -37.80687844, 145.06270..."
56,CROYDON SOUTH,"POLYGON ((145.29646275 -37.81753846, 145.29449..."
59,COBURG NORTH,"POLYGON ((144.98314404 -37.73597902, 144.98305..."


In [9]:
# Defining a function which will return the suburb based on the point formed using longitude and latitude

def get_suburb(row):
    p = Point(row['Longitude'],row['Latitude'])   # Creating a point using longitude and latitude
    # Iterating over each row in shape_df
    for index, item in shape_df.iterrows():
        # Checking and fetching the suburb to which the point belongs
        if shape(item["geometry"]).contains(p):
            return item['VIC_LOCA_2']

In [15]:
cityOfMel_df = pd.read_csv("City of Melbourne Childcare_centres.csv")

In [17]:
cityOfMel_df['Suburb'] = cityOfMel_df.apply(get_suburb, axis=1)  # Applying the function to get data in Suburb column 

In [18]:
cityOfMel_df.head()

,ref,name,contact_ph,url,Latitude,Longitude,Suburb
0,106080,RMIT City Campus Children's Centre,03 8663 4810,NaN,-37.806387,144.965787,CARLTON
1,664202,One Tree Defence Childcare Unit Coventry,03 9282 5009,www.onetree.org.au,-37.828957,144.969682,SOUTHBANK
2,617536,Hotham Hub Children's Centre,03 9658 8812,NaN,-37.792711,144.942022,NORTH MELBOURNE
3,627399,Only About Children,03 9922 1170,NaN,-37.810462,144.962384,MELBOURNE
4,592707,Goodstart Early Learning Melbourne Flinders St...,03 9671 3539,www.goodstart.org.au,-37.817946,144.965286,MELBOURNE


In [48]:
cityOfMel_df.to_csv('cityOfMelbourne-with Suburb.csv',index=False)

### Filtering Bilingual/Multicultural Childcare Centers from the dataset Australia National Registers Education-services-au-export.csv

In [7]:
import re

In [21]:
aus_df = pd.read_csv("Education-services-au-export.csv")
aus_df.head()

,ServiceApprovalNumber,Provider Approval Number,ServiceName,ProviderLegalName,ServiceType,ServiceAddress,Suburb,State,Postcode,Phone,...,Holiday Care Wednesday End Time,Holiday Care Thursday Start Time,Holiday Care Thursday End Time,Holiday Care Friday Start Time,Holiday Care Friday End Time,Holiday Care Saturday Start Time,Holiday Care Saturday End Time,Holiday Care Sunday Start Time,Holiday Care Sunday End Time,Temporarily Closed
0,SE-00000002,PR-00000898,Community Kids Waterford Early Education Centre,G8 Education Limited,Centre-Based Care,8-10 Allora St,WATERFORD WEST,QLD,4133.0,0732009852,...,18:15,06:15,18:15,06:15,18:15,NaN,NaN,NaN,NaN,No
1,SE-00000003,PR-40010446,Pelican Point Early Learning Centre Eagleby,GESHWEND FAMILY PTY LTD,Centre-Based Care,82 Fryar Road,EAGLEBY,QLD,4207.0,0738073222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
2,SE-00000004,PR-00000898,Community Kids - Broadbeach Waters,G8 Education Limited,Centre-Based Care,4 Fremar Street,BROADBEACH WATERS,QLD,4217.0,0755920306,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
3,SE-00000005,PR-00000054,Burleigh Waters Child Care Centre,Downlip Pty Ltd & Asian Business Connections P...,Centre-Based Care,143 Christine Ave,BURLEIGH WATERS,QLD,4220.0,0755353288,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
4,SE-00000006,PR-00000483,Small World Child Care Centre,Kika Massouras,Centre-Based Care,113-115 Pappas Way,CARRARA,QLD,4211.0,07 5594 1844,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No


In [47]:
aus_df[aus_df.ServiceName.str.contains('.*[Ll]ingu.*|.*[Mm]ulticultur.*|.*[Ll]anguage.*|[Cc]hinese.*')].to_csv('regex_filter.csv',index=False)

### Fixing phone numbers by removing spaces

In [23]:
childcare = pd.read_csv("childcare_final.csv",  encoding = "ISO-8859-1")

In [24]:
childcare

,centre_name,provide_name,centre_type,centre_address,suburb,state,postcode,phone,email,educational_program_and_practice,...,part_of_school,standalone,after_school,before_school,vacation_care,website,fees,activities,language,aim
0,John Street Community Early Childhood Co-opera...,John Street Community Early Childhood Co-op Ltd,Centre-Based Care,23 John Street,FITZROY,VIC,3065,03 9419 6659,admin@johnst.com.au,Exceeding NQS,...,No,No,No,No,No,www.johnst.com.au/,140,Weekly Music and Movement session and speciali...,The educators at John Street are eager to lear...,We design our childrens programs to be comple...
1,East West Childcare,East West Childcare Association Inc,Centre-Based Care,138 George Street,FITZROY,VIC,3065,394191673,admin@eastwestck.org.au,Exceeding NQS,...,No,No,No,No,No,www.eastwestck.org.au/,139,The kindergarten program also enables us to fu...,NaN,We are a co-operative of parents and early chi...
2,Acacia Fitzroy Creche,Acacia Indo-chinese Community Support Associat...,Centre-Based Care,187 Napier Street,FITZROY,VIC,3065,394191142,afc@acacia.org.au,Meeting NQS,...,No,No,No,No,No,www.acacia.org.au/acacia-fitzroy-creche/,NaN,All programs provide innovating rich indoor/ou...,NaN,Our educational programs reflect the full appr...
3,Indra Preschool,Indra Preschool Association Inc,Centre-Based Care,38 Edinburgh Road,BLACKBURN SOUTH,VIC,3130,398773640,indra.kin@kindergarten.vic.gov.au,Exceeding NQS,...,No,Yes,No,No,No,indrapreschool.vic.edu.au/,2480 per year,Throughout the year we focus on assisting each...,NaN,We believe children are curious and we provid...
4,Starfish Early Learning Centre,Starfish ChildCare Pty Ltd T/as Starfish Early...,Centre-Based Care,168 Canterbury Road,BLACKBURN SOUTH,VIC,3130,398773930,info@starfishelc.com.au,Exceeding NQS,...,No,No,No,No,No,www.starfishelc.com.au/,NaN,We strive to provide a curriculum that provide...,NaN,We are committed to providing opportunities of...
5,Bright Horizons Australia Childcare Croydon North,Bright Horizons Australia Childcare Pty Ltd,Centre-Based Care,156-158 Yarra Road,CROYDON HILLS,VIC,3136,03 9723 0588,croydonnorth@bhchildcare.com.au,Meeting NQS,...,No,No,No,No,No,www.bhchildcare.com.au/,113,"Facilitate their involvement in small group, w...",NaN,"Through our curriculum, we aim to provide a fr..."
6,Young Einsteins ELC Clayton,Young Einsteins Early Learning Centre Pty Ltd ...,Centre-Based Care,1 Milton Avenue,CLAYTON SOUTH,VIC,3169,395435039,info@youngeinsteins.com.au,Meeting NQS,...,No,No,No,No,Yes,www.youngeinsteins.com.au/,NaN,Our Curriculum will cater to the individual ne...,NaN,Our aim is to provide a Pre-Kindi and Kinderga...
7,Good Shepherd Lutheran OSHClub,OSHCLUB PTY LTD,Centre-Based Care,53-57 Plymouth Road,CROYDON HILLS,VIC,3136,423777355,jagvicsa@junioradventuresgroup.com.au,Meeting NQS,...,No,No,Yes,Yes,Yes,www.oshclub.com.au/good-shepherd-lutheran-vic/...,NaN,Our Coordinators plan a wide variety of struct...,NaN,Building brighter futures together in partners...
8,Clarinda Primary School Out of School Hours Care,Kingston City Council,Centre-Based Care,1166 Centre Road,CLAYTON SOUTH,VIC,3169,03 95814847,beforeandafterschoolprogram@kingston.vic.gov.au,Meeting NQS,...,No,No,Yes,Yes,No,www.clarindaps.vic.edu.au/about-us/school-hour...,NaN,Sessions offer a mix of structured and unstruc...,NaN,Kingston Council is proud to partner with Clar...
9,Holbury Children's Centre,Holbury Children's Centre Inc,Centre-Based Care,29 Raleigh Street,BLACKBURN SOUTH,VIC,3130,398783656,janine@holbury.org.au,Exceeding NQS,...,No,No,No,No,No,www.holbury.org.au/,NaN,The program and activities provide early learn...,NaN,Our 3 and 4 year old kindergarten promotes ind...


In [25]:
childcare['phone'].str.replace(' ','')

0     0394196659
1      394191673
2      394191142
3      398773640
4      398773930
5     0397230588
6      395435039
7      423777355
8     0395814847
9      398783656
10    1300035000
11     398086076
12     394261515
13     388789878
14     395514914
15     455328828
16     395510870
17     395813057
18     395400333
19     383901133
20     386829400
21     395274343
22     399004648
23     380808041
24     394282282
25     392742740
26     394291488
27     394296150
28     394283433
29    0394282663
30     394286537
31     395258313
32     395275963
33     395275168
34     395228227
35    0394292944
36     394266061
37    0394295985
38     477882819
39    0394259217
40     392055479
41    1300072410
42     399945611
43    1300366437
44     390461925
45     386523270
46     395275114
47     394283113
48           NaN
Name: phone, dtype: object

In [28]:
childcare['phone'] = childcare['phone'].str.replace(' ','').replace("(^3)", "03",regex=True)

In [29]:
childcare.head()

,centre_name,provide_name,centre_type,centre_address,suburb,state,postcode,phone,email,educational_program_and_practice,...,part_of_school,standalone,after_school,before_school,vacation_care,website,fees,activities,language,aim
0,John Street Community Early Childhood Co-opera...,John Street Community Early Childhood Co-op Ltd,Centre-Based Care,23 John Street,FITZROY,VIC,3065,0394196659,admin@johnst.com.au,Exceeding NQS,...,No,No,No,No,No,www.johnst.com.au/,140,Weekly Music and Movement session and speciali...,The educators at John Street are eager to lear...,We design our childrens programs to be comple...
1,East West Childcare,East West Childcare Association Inc,Centre-Based Care,138 George Street,FITZROY,VIC,3065,0394191673,admin@eastwestck.org.au,Exceeding NQS,...,No,No,No,No,No,www.eastwestck.org.au/,139,The kindergarten program also enables us to fu...,NaN,We are a co-operative of parents and early chi...
2,Acacia Fitzroy Creche,Acacia Indo-chinese Community Support Associat...,Centre-Based Care,187 Napier Street,FITZROY,VIC,3065,0394191142,afc@acacia.org.au,Meeting NQS,...,No,No,No,No,No,www.acacia.org.au/acacia-fitzroy-creche/,NaN,All programs provide innovating rich indoor/ou...,NaN,Our educational programs reflect the full appr...
3,Indra Preschool,Indra Preschool Association Inc,Centre-Based Care,38 Edinburgh Road,BLACKBURN SOUTH,VIC,3130,0398773640,indra.kin@kindergarten.vic.gov.au,Exceeding NQS,...,No,Yes,No,No,No,indrapreschool.vic.edu.au/,2480 per year,Throughout the year we focus on assisting each...,NaN,We believe children are curious and we provid...
4,Starfish Early Learning Centre,Starfish ChildCare Pty Ltd T/as Starfish Early...,Centre-Based Care,168 Canterbury Road,BLACKBURN SOUTH,VIC,3130,0398773930,info@starfishelc.com.au,Exceeding NQS,...,No,No,No,No,No,www.starfishelc.com.au/,NaN,We strive to provide a curriculum that provide...,NaN,We are committed to providing opportunities of...


In [38]:
childcare.to_csv("childcare_final.csv", index=False) 